# Parsing Julia to Tapenade's IL

This notebook shows how to parse julia code into the format used by tapenade in a way to start developping source transform adjoint code. 

## Load Tapenade's tree structure
Start with compiling

In [3]:
#= 
Add some info and some calls to the makefile implemented in this example
=#

Load the newly created library

In [4]:
using Libdl
c_libname = "./treelib.so"
lib = Libdl.dlopen(c_libname) # Open the library explicitly.

Ptr{Nothing} @0x0000000002089640

Define function handler to useful functions.

In [5]:
c_newTreeBuilder = Libdl.dlsym(lib, :newTreeBuilder)   # Get a symbol for the function to call.
c_startTree = Libdl.dlsym(lib, :startTree)
c_turnListFrontier = Libdl.dlsym(lib, :turnListFrontier)
c_startAnnotation = Libdl.dlsym(lib, :startTree) ### This is an extern function, and we might want to have this pointing to another lib!
c_putValue = Libdl.dlsym(lib, :putValue) ## Also extern
c_putTree = Libdl.dlsym(lib, :putTree) ## Also extern
c_oneLessWaiting = Libdl.dlsym(lib, :oneLessWaiting) 
c_putListTree = Libdl.dlsym(lib, :putListTree) ## Extern
c_startDummyTree = Libdl.dlsym(lib, :startDummyTree) ## Extern
c_terminateListTree = Libdl.dlsym(lib, :terminateListTree) ## Extern
c_getTreeBuilt = Libdl.dlsym(lib, :getTreeBuilt) ## Extern
c_getListTreeBuilt = Libdl.dlsym(lib, :getListTreeBuilt) ## Extern
c_deleteTreeBuilder = Libdl.dlsym(lib, :deleteTreeBuilder) ## Extern
c_resetTreeBuilder = Libdl.dlsym(lib, :resetTreeBuilder) ## Extern
c_removeTree = Libdl.dlsym(lib, :removeTree)
c_showTreeBuilderState = Libdl.dlsym(lib, :showTreeBuilderState) ## Debug purposes

Ptr{Nothing} @0x00007fdb50878bfd

## Create the associated structures on Julia's side

In [6]:
struct BinaryTree
    v::Cfloat
    lchild::Ptr{BinaryTree}
    rchild::Ptr{BinaryTree}
end

In [7]:
using DataStructures

In [8]:
ct_testfile = "ct_test.jl"
orig_testfile = "kepler.jl"
basic_test = "testfile.jl"

"testfile.jl"

In [9]:
EndOfList = :EndOfList
cstSymb = :cst

:cst

In [10]:
metalnames = Dict(
    :(=) => "assign",
    :. => "package", # Check whether the dot refers to a local path or is synonym for an import
    :block => "block",
    :vect => "vect",
    :vcat => "vcat",
    :return => "return",
    :tuple => "tuple",
    :ref => "ref",
    :curly => "curly",
    :call => "call", 
    :function => "function",
    :using => "using", 
    :toplevel => "toplevel",
    Int64 => "int64",
    Float64 => "float64",
    Int32 => "int32",
    Float32 => "float32",
    LineNumberNode => "linenumber",
    Nothing => "none"
)

Dict{Any, String} with 19 entries:
  :return        => "return"
  :vect          => "vect"
  Nothing        => "none"
  :tuple         => "tuple"
  Int64          => "int64"
  :.             => "package"
  :ref           => "ref"
  Int32          => "int32"
  :curly         => "curly"
  LineNumberNode => "linenumber"
  :call          => "call"
  :vcat          => "vcat"
  :block         => "block"
  Float64        => "float64"
  :function      => "function"
  :(=)           => "assign"
  :using         => "using"
  Float32        => "float32"
  :toplevel      => "toplevel"

In [11]:
typeof(EndOfList)

Symbol

In [14]:
function file2ast(fname::AbstractString, outfile::AbstractString)
    all_symbols = Set()
    open(outfile, "w") do out 
        code = read(fname, String)
        ## println(code)

        exp_in_waiting = Stack{Any}()
        
        prev = 0
        next = 1
        while next > prev 
            prev = next
            exp, next = Meta.parse(code, prev)
            push!(exp_in_waiting, exp)
            ## println("     ***** Currently looking at: ")
            ## println(exp)
            while !isempty(exp_in_waiting)
                e = pop!(exp_in_waiting)
                if e isa Expr 
                    # Parse expression
                    # Write current instruction
                    println("     ***** Currently looking at: ", string(e))
                    println(e.head)
                    println("Operator ", e.head, " has ", length(e.args), " children. ")
                    println(e.args)
                    # If Current instruction is a call, make sure to add an end of list
                    # Same thing holds for a 'using' statement
                    #####################
                    # NOTE:             #
                    # One might need    #
                    # to check if other #
                    # keywords need this#
                    #####################
                    if e.head in (:call, :tuple, :using, :vect) 
                        push!(exp_in_waiting, EndOfList)
                    end
                    for arg in Iterators.reverse(e.args)
                        push!(exp_in_waiting, arg)
                    end
                    push!(all_symbols, metalnames[e.head]) 
                    push!(exp_in_waiting, e.head) 
                    # write(out, e.head) # New C node from tree builder
                    # write(out, "\n")
                elseif e isa Symbol
                    # Add symbol to file
                    if haskey(metalnames, e) 
                        write(out, metalnames[e]) # New leaf for current operator
                    else
                        write(out, e) # New leaf for current operator
                    end
                    write(out, "\n")
                else
                    # Neither a symbol nor an expression, it's probably a constant
                    ## println("Type of our constant thing is ", typeof(e))
                    push!(all_symbols, metalnames[typeof(e)])
                    write(out, metalnames[typeof(e)])
                    write(out, "\n")
                    ## println("Looking at cst things")
                    ## println(e)
                    write(out, string(e))
                    write(out, "\n")
                end
            end
                    
            
        end
    end
    return all_symbols
end

file2ast (generic function with 1 method)

In [15]:
all_symbols = file2ast(basic_test, "outputtest")

     ***** Currently looking at: using LinearAlgebra, ForwardDiff
using
Operator using has 2 children. 
Any[:($(Expr(:., :LinearAlgebra))), :($(Expr(:., :ForwardDiff)))]
     ***** Currently looking at: $(Expr(:., :LinearAlgebra))
.
Operator . has 1 children. 
Any[:LinearAlgebra]
     ***** Currently looking at: $(Expr(:., :ForwardDiff))
.
Operator . has 1 children. 
Any[:ForwardDiff]
     ***** Currently looking at: Tmax = 6
=
Operator = has 2 children. 
Any[:Tmax, 6]
     ***** Currently looking at: cTmax = 3600 ^ 2 / 1.0e6
=
Operator = has 2 children. 
Any[:cTmax, :(3600 ^ 2 / 1.0e6)]
     ***** Currently looking at: 3600 ^ 2 / 1.0e6
call
Operator call has 3 children. 
Any[:/, :(3600 ^ 2), 1.0e6]
     ***** Currently looking at: 3600 ^ 2
call
Operator call has 3 children. 
Any[:^, 3600, 2]
     ***** Currently looking at: mass0 = 1500
=
Operator = has 2 children. 
Any[:mass0, 1500]
     ***** Currently looking at: β = 0.0142
=
Operator = has 2 children. 
Any[:β, 0.0142]
     ***** C

Set{Any} with 17 elements:
  "call"
  "vcat"
  "tuple"
  "using"
  "linenumber"
  "package"
  "float64"
  "assign"
  "int64"
  "curly"
  "block"
  "ref"
  "toplevel"
  "function"
  "none"
  "vect"
  "return"